This project is a data analysis tool designed to provide insights into vehicle data. It includes displaying visualizations using Plotly Express. The tool allows users to explore the distribution of vehicle prices, analyze the relationship between various factors, and toggle the visibility of specific visualizations.
The aim of this project is to perform Exploratory Data Analysis (EDA) on the given dataset.

In [10]:
#Importing Libraries
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px

In [12]:
df = pd.read_csv(r'C:\Users\tevin\SDTProject\vehicles_us.csv')

In [13]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [14]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [15]:
df.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


#### Removing Nan values in the model year, model and odometer column.

In [16]:
# Remove rows with NaN values in'model_year', and 'model' columns in the original DataFrame
df.dropna(subset=['model_year', 'model','odometer'], inplace=True)
#Create the Make column
df['make'] = df['model'].str.split(' ').str[0]

#Update maodel column
df['model'] = df['model'].str.split(' ').str[1]


From the above code, a new column was created called make and model column was modified to only have models.

In [17]:
df['is_4wd'] = df['is_4wd'].fillna(0)

# Convert 'is_4wd' and 'model_year' columns to integer type
df[['is_4wd', 'model_year']] = df[['is_4wd', 'model_year']].astype(int)

Assuming that 1 is Yes and 0 is no, all nan values were changed to 0 and then the columns was changed to int type.

In [18]:
grouped_df = df.groupby(['make', 'type']).size().reset_index(name='count')

# Replace NaN values in 'type' column with a default value
grouped_df['type'] = grouped_df['type'].fillna('Unknown')

# Plotly Express bar chart
fig = px.bar(grouped_df, x='make', y='count', color='type', title='Vehicle Count by Manufacturer with Type Selection',
             labels={'make': 'Manufacturer', 'count': 'Count', 'type': 'Vehicle Type'},
             category_orders={'make': sorted(df['make'].unique())})

# Show the Plotly Express chart
fig.show()

c:\Users\tevin\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


From the graph above we see that SUV's and sedans are the best selling model type. 

In [19]:
# mileage, price, and vehicle types
fig = px.scatter(df, x='odometer', y='price', color='make', title='Scatter Plot: Mileage vs. Price', labels={'odometer': 'Mileage', 'price': 'Price', 'make': 'Vehicle Make'})

# Add a dropdown menu for selecting vehicle types
fig.update_layout(updatemenus=[dict(type='buttons', showactive=True, buttons=[
    dict(label='All Make', method='update', args=[{'visible': [True] * len(df['make'].unique())}]),
    *[
        dict(label=vehicle_type, method='update', args=[{'visible': [type == vehicle_type for type in df['make'].unique()]}])
        for vehicle_type in df['make'].unique()
    ]
])])

# Show the plot
fig.show()

c:\Users\tevin\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



The graph above shows that vehicles with more mileage tend to have a lower price. This is true for all make.

In [20]:
# vehicle model years and vehicle conditions
fig = px.histogram(df, x='model_year', color='condition', title='Distribution of Model Year by Condition', labels={'model_year': 'Model Year', 'condition': 'Condition'})

# Show the plot
fig.show()

c:\Users\tevin\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



From the graph above it shows that the younger the vehicle is the better the condition is.

In [21]:
# vehicle makes and vehicle prices
average_price_by_make = df.groupby('make')['price'].mean().reset_index()

# Create a bar chart with different colors for each make
fig = px.bar(average_price_by_make, x='make', y='price', color='make', title='Average Price by Vehicle Make', labels={'make': 'Make', 'price': 'Average Price'})

# Show the plot
fig.show()

c:\Users\tevin\anaconda3\Lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



From the graph above by make, the highest average price is from Ram and Cadillac

In [22]:
# Assuming 'price' is the column for vehicle prices
fig = px.histogram(df, x='price', title='Distribution of Vehicle Prices', labels={'price': 'Price'})

# Show the plot
fig.show()

From the graph above, majority of the price is 20k or less. The data is skewed right, only a few vehicles are above 50k.

## Conclusion

Our exploratory data analysis (EDA) has provided valuable insights into the dynamics of the dataset, shedding light on key trends and patterns within the automotive market. Here's a summary of the noteworthy findings:

Best-Selling Model Types:
- The distribution of vehicle types reveals that SUVs and sedans emerge as the dominant players in the market. These types significantly outpace others in terms of sales.

Mileage vs. Price:
- A compelling inverse relationship is observed between mileage and price across all vehicle makes. Vehicles with higher mileage tend to be priced lower, indicating a consistent market trend.

Vehicle Age vs. Condition:
- The boxplot analysis suggests a positive correlation between the age of a vehicle and its condition. Younger vehicles generally exhibit better conditions, reinforcing the expectation that newer models are better maintained.

Highest Average Prices by Make:
- When examining average prices by make, two standout performers are Ram and Cadillac. These brands command the highest average prices in the market.
Price Distribution:

The histogram illustrates a right-skewed distribution of prices, with the majority of vehicles priced at $20,000 or less. Notably, only a limited number of vehicles surpass the $50,000 mark, indicating a concentration of prices in the lower range.
In conclusion, these insights provide a comprehensive understanding of market dynamics, aiding decision-makers in the automotive industry. Understanding the popularity of certain types, the impact of mileage on pricing, and the influence of vehicle age on condition are pivotal considerations for various stakeholders. These findings can inform marketing strategies, inventory management, and pricing decisions, ultimately contributing to a more informed and data-driven approach within the automotive sector.